# Small Blackjack

We consider a reduced version of **blackjack**, which we call **small blackjack**. Like in the original game of blackjack, a **player** plays against the **dealer**. Both agents (**player** and **dealer**) start with a single card. The **player** first take his turns, and then (once the player is finished) the **dealer** may play (unless the player was _busted_, in which case the dealer authomatically wins, and need not play).

In each turn, an agent may choose two actions: **HIT** or **HOLD**. By choosing **HIT**, the agent is asking for one more card from the deck. By choosing **HOLD**, the agent finishes his turn without drawing an additional card.

The agent **total** is the sum of **values** of all cards the agent has drawn, including the initial card handed to it on the start of the game. The biggest difference between the **small blackjack** and the original one is that the cards are counted differently. The "small cards" (2, 3, 4, ..., 9) all have the value equal to 5. The "face cards" (A, J, Q, K) and 10 all have value 10. In this version of the game, the ACE has no special meaning. It is just another "face card".

In the original game, the goal is to have the highest total not exceeding 21. However, since 21 is actually impossible to attain in this version, **THE GOAL IS TO HAVE THE HIGHEST TOTAL NOT BIGGER THAN 20**. If an agent achieves the total bigger than 21 it losses the game (we say that the agent is _busted_). More precisely:

* If the player is busted, dealer wins.
* If the players is not busted, then:
  - If the dealer is busted, player wins.
  - If the dealer is not busted, the agent with the highest total wins.
  - In case that neither agent is busted, and both have the same total, the game is draw.
 
Upon playing each game, the wining agent receives reward +1, the losing agent receives -1. If the game is draw, both agents receive 0. The reward is given only when the game is over. No reward is received after non-terminating actions (the reward following those actions is 0).

<div class="alert alert-block alert-info">
<b>The Assignment:</b> 
Devise an optimal strategy for playing small blackjack.
</div>

<div class="alert alert-block alert-warning">
<b>Dealer strategy:</b> 
    Assume that the dealer is playing according to a fixed strategy: Choose HIT as long as the dealer total is not 15 or 20, otherwise choose HOLD.
</div>

## Basic Imports

In [1]:
from enum import Enum
from dataclasses import dataclass, astuple
from typing import Callable
from random import random, randint
from copy import copy, deepcopy

from tqdm import trange
from rich import print

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Cards and Decks of Cards

Let us first implement cards and decks of cards.

In [2]:
class CardType(Enum):
    """An enumeration representing card types."""

    SMALL = 5
    BIG = 10

    def __repr__(self):
        return str(self.value)

In [3]:
def draw_probability(card: CardType):
    if card == CardType.SMALL:
        return 8*4/52
    else:
        return 5*4/52

## Turns and Turn States

In [4]:
def update_total(total: int, card: CardType) -> int | None:
    """Update agent's total by taking into account a newly drawn card.

    If the player's new total is bigger than 20, and the player is busted,
    the function returns `None`.
    
    Args:
        total (int): player total
        has_ace (bool): indicates if the player has usable ACE
        card_value (CardValue): value of the newly drawn card

    Returns:
        int | None: updated player total, or `None` if the player is busted
    """
    total += card.value
    if total > 20:
        return None
    else:
        return total

In [5]:
class TurnStateType(Enum):
    INITIAL = 0
    PLAYING = 1
    TERMINAL = 2

    def __repr__(self):
        if self == TurnStateType.INITIAL:
            return "I"
        elif self == TurnStateType.PLAYING:
            return "P"
        elif self == TurnStateType.TERMINAL:
            return "T"

In [6]:
class TurnStateValue(Enum):
    FIVE = 5
    TEN = 10
    FIFTEEN = 15
    TWENTY = 20
    BUSTED = 25

    def __repr__(self):
        return str(self.value)

In [7]:
@dataclass
class TurnState:
    type: TurnStateType
    value: TurnStateValue

    def __repr__(self):
        if self.value == TurnStateValue.BUSTED:
            return "B"
        elif self.type == TurnStateType.PLAYING:
            return repr(self.value)
        else:
            return f"{repr(self.type)}{repr(self.value)}"
            

In [53]:
def turn_states() -> list[TurnState]:
    initial_states = [
        TurnState(TurnStateType.INITIAL, TurnStateValue.FIVE),
        TurnState(TurnStateType.INITIAL, TurnStateValue.TEN)]
    playing_states = [TurnState(TurnStateType.PLAYING, v) for v in iter(TurnStateValue) if v != TurnStateValue.BUSTED]
    terminal_states = [TurnState(TurnStateType.TERMINAL, v) for v in iter(TurnStateValue)]
    return initial_states + playing_states + terminal_states

In [54]:
turn_states()

[I5, I10, 5, 10, 15, 20, T5, T10, T15, T20, B]

## Actions

In [55]:
class Action(Enum):
    HIT = 0
    HOLD = 1

    def __repr__(self):
        if self == Action.HIT:
            return "HIT"
        else:
            return "HOLD"

## Modelling a Single Turn

In [56]:
def turn_transition_array():
    states = turn_states()
    transition_array = np.zeros(shape=(len(states), len(states), 2))
    for i, s in enumerate(states):
        if s.type == TurnStateType.INITIAL:
            j = states.index(TurnState(TurnStateType.PLAYING, s.value))
            transition_array[i, j, 0] = 1
            transition_array[i, j, 1] = 1
        elif s.type == TurnStateType.PLAYING:
            # HOLD: PLAYING state => TERMINAL state
            j = states.index(TurnState(TurnStateType.TERMINAL, s.value))
            transition_array[i, j, 1] = 1
            # HIT: The following state depend on the card drawn
            for card in iter(CardType):
                next_total = update_total(s.value.value, card)
                if next_total is None:
                    j = states.index(TurnState(TurnStateType.TERMINAL, TurnStateValue.BUSTED))
                else:
                    j = states.index(TurnState(TurnStateType.PLAYING, TurnStateValue(next_total)))
                transition_array[i, j, 0] += draw_probability(card)
        else:
            transition_array[i, i, :] = 1
    return transition_array

In [57]:
def print_transition_matrix(ta, ndx=None):
    states = [str(s) for s in turn_states()]
    tm = ta if ndx is None else ta[:, :, ndx]
    return pd.DataFrame(tm, columns=states, index=states)

In [58]:
TTA = turn_transition_array()

In [59]:
print_transition_matrix(TTA, 0)

,I5,I10,5,10,15,20,T5,T10,T15,T20,B
I5,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
I10,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.0,0.615385,0.384615,0.000000,0.0,0.0,0.0,0.0,0.000000
10,0.0,0.0,0.0,0.000000,0.615385,0.384615,0.0,0.0,0.0,0.0,0.000000
15,0.0,0.0,0.0,0.000000,0.000000,0.615385,0.0,0.0,0.0,0.0,0.384615
20,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000
T5,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000
T10,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.000000
T15,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.000000
T20,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.000000


In [60]:
print_transition_matrix(TTA, 1)

,I5,I10,5,10,15,20,T5,T10,T15,T20,B
I5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
T5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
T10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
T15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
T20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [61]:
# Starting from any init state, the total probability of landing in any of other states must be 1 for all actions
pd.DataFrame(np.sum(TTA, axis=1), columns=[str(a) for a in iter(Action)], index=[str(s) for s in turn_states()])

,Action.HIT,Action.HOLD
I5,1.0,1.0
I10,1.0,1.0
5,1.0,1.0
10,1.0,1.0
15,1.0,1.0
20,1.0,1.0
T5,1.0,1.0
T10,1.0,1.0
T15,1.0,1.0
T20,1.0,1.0


## Predicting Terminal State Probabilities Given a Policy and an Initial State

In [62]:
def dealer_policy(s):
    if s.type != TurnStateType.TERMINAL and s.value.value <= 15:
        return [1, 0]
    else:
        return [0, 1]

In [63]:
def build_policy_matrix(policy):
    states = turn_states()
    policy_matrix = np.zeros(shape=(len(states), 2))
    for i, s in enumerate(states):
        policy_matrix[i, :] = policy(s)
    return policy_matrix

In [64]:
POLICY_MATRIX = build_policy_matrix(dealer_policy)
pd.DataFrame(POLICY_MATRIX, columns=[str(a) for a in iter(Action)], index=[str(s) for s in turn_states()])

,Action.HIT,Action.HOLD
I5,1.0,0.0
I10,1.0,0.0
5,1.0,0.0
10,1.0,0.0
15,1.0,0.0
20,0.0,1.0
T5,0.0,1.0
T10,0.0,1.0
T15,0.0,1.0
T20,0.0,1.0


In [65]:
def build_policy_transition_matrix(transition_array, policy):
    policy_matrix = build_policy_matrix(policy)
    states_no = policy_matrix.shape[0]
    ptm = np.zeros(shape=(states_no, states_no))
    for i in range(states_no):
        ptm[i, :] = transition_array[i, :, 0] * policy_matrix[i, 0] + transition_array[i, :, 1] * policy_matrix[i, 1]
    return ptm

In [66]:
POLICY_TRANSITION_MATRIX = build_policy_transition_matrix(TTA, dealer_policy)
print_transition_matrix(POLICY_TRANSITION_MATRIX)

,I5,I10,5,10,15,20,T5,T10,T15,T20,B
I5,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
I10,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.0,0.615385,0.384615,0.000000,0.0,0.0,0.0,0.0,0.000000
10,0.0,0.0,0.0,0.000000,0.615385,0.384615,0.0,0.0,0.0,0.0,0.000000
15,0.0,0.0,0.0,0.000000,0.000000,0.615385,0.0,0.0,0.0,0.0,0.384615
20,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.000000
T5,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000
T10,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.000000
T15,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.000000
T20,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.000000


In [67]:
pd.DataFrame(np.sum(POLICY_TRANSITION_MATRIX, axis=1), index=[str(s) for s in turn_states()], columns=["total"])

,total
I5,1.0
I10,1.0
5,1.0
10,1.0
15,1.0
20,1.0
T5,1.0
T10,1.0
T15,1.0
T20,1.0


## Playing with Graphs

In [68]:
def input_nodes(adjacency_matrix, node):
    inputs = []
    for i in range(adjacency_matrix.shape[0]):
        prob = adjacency_matrix[i, node]
        if prob != 0:
            inputs.append((i, prob))
    return inputs

In [69]:
def output_nodes(adjacency_matrix, node):
    outputs = []
    for i in range(adjacency_matrix.shape[0]):
        prob = adjacency_matrix[node, i]
        if prob != 0:
            outputs.append((i, prob))
    return outputs

In [70]:
input_nodes(POLICY_TRANSITION_MATRIX, 3)

[(1, 1.0), (2, 0.6153846153846154)]

In [71]:
output_nodes(POLICY_TRANSITION_MATRIX, 3)

[(4, 0.6153846153846154), (5, 0.38461538461538464)]

In [72]:
def get_visit_probabilities(prob5):    
    probs = np.random.rand(POLICY_TRANSITION_MATRIX.shape[0])
    probs[0] = prob5
    probs[1] = 1 - prob5
    for i in range(len(probs)):
        if i not in [0, 1]:
            inputs = input_nodes(POLICY_TRANSITION_MATRIX, i)
            update = 0
            for j, p in inputs:
                if i != j:
                    update += probs[j]*p
            probs[i] = update
    return probs

In [73]:
def get_all_visit_probabilities():
    p05 = get_visit_probabilities(1)
    p10 = get_visit_probabilities(0)
    return np.transpose(np.vstack([p05, p10]))

In [74]:
probs = get_all_visit_probabilities()
pd.DataFrame(probs, index=[str(s) for s in turn_states()], columns=["5", "10"])

,5,10
I5,1.000000,0.000000
I10,0.000000,1.000000
5,1.000000,0.000000
10,0.615385,1.000000
15,0.763314,0.615385
20,0.706418,0.763314
T5,0.000000,0.000000
T10,0.000000,0.000000
T15,0.000000,0.000000
T20,0.706418,0.763314


## Generating the model

In [117]:
MODEL = np.zeros(shape=(len(TURN_STATES), 3, len(TURN_STATES), 2))

In [118]:
# POPULATING RESULTS FOR ACTION
def reward(v):
    return v+1

def is_busted(s):
    return s == 6

for a in [0, 1]:
    for s, (total, init) in enumerate(TURN_STATES):
        outputs = output_nodes(TRANSITION_MATRIX[:, :, a], s)
        if init:
            s_next = TURN_STATES.index((total, False))
            MODEL[s_next, reward(0), s, a] = 1
        else:
            for s_next, prob in outputs:
                if a == 0: # HIT
                    r = reward(0 if not is_busted(s_next) else -1)
                    MODEL[s_next, r, s, a] += prob
                else: # HOLD
                    MODEL[s_next, reward(1), s, a] += prob*terminal_probs[6] # If dealer is busted player wins
                    MODEL[s_next, reward(1), s, a] += prob*sum([p for i, p in enumerate(terminal_probs) if p < s and i != 6]) # If dealer has lower score, player wins
                    MODEL[s_next, reward(0), s, a] += prob*sum([p for i, p in enumerate(terminal_probs) if p == s and i != 6]) # If the scores are equal, draw
                    MODEL[s_next, reward(-1), s, a] += prob*sum([p for i, p in enumerate(terminal_probs) if p > s and i != 6]) # If dealer has lower score, player loses

In [119]:
pd.DataFrame(MODEL[:, 1, :, 0], index=STATE_NAMES, columns = STATE_NAMES)

,i5,i10,5,10,15,20,B
i5,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
i10,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
5,1.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
10,0.0,1.0,0.615385,0.000000,0.000000,0.0,0.0
15,0.0,0.0,0.384615,0.615385,0.000000,0.0,0.0
20,0.0,0.0,0.000000,0.384615,0.615385,0.0,0.0
B,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [120]:
pd.DataFrame(MODEL[:, 0, :, 0], index=STATE_NAMES, columns = STATE_NAMES)

,i5,i10,5,10,15,20,B
i5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
i10,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
10,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
15,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
20,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
B,0.0,0.0,0.0,0.0,0.384615,1.0,0.0


In [121]:
pd.DataFrame(MODEL[:, 2, :, 0], index=STATE_NAMES, columns = STATE_NAMES)

,i5,i10,5,10,15,20,B
i5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
i10,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
pd.DataFrame(MODEL[:, 1, :, 1], index=STATE_NAMES, columns = STATE_NAMES)

,i5,i10,5,10,15,20,B
i5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
i10,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,1.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
pd.DataFrame(MODEL[:, 0, :, 1], index=STATE_NAMES, columns = STATE_NAMES)

,i5,i10,5,10,15,20,B
i5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
i10,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
pd.DataFrame(MODEL[:, 2, :, 1], index=STATE_NAMES, columns = STATE_NAMES)

,i5,i10,5,10,15,20,B
i5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
i10,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,1.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,1.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0
